# Starter code for measuring angular separations on the sky

## Learning goals
The purpose of this noteobok is to familiarize yourself with reading, plotting, and determining the statistics of fits images from the 30inch telescope. As a secondary goal we will also impplemenet a *VERY BASIC*  CCD Reduction (without many bells and whistles that will be needed when we do a final data reduction). 

We will start by loading in the nessisary libraries. Inside the libraries exists defintions (functions) that allow us to carry out what we want the code to do. We will also then load in the data set needed 

Lets begin!

## Data Location : 
UTBox  [256MB]: https://utexas.box.com/s/728urjnexttq1v6jdkntp3raeb89mz3w

## Loading Libraries 

In [ ]:
#Import libraries 
"""

"""
#remove if you dont have a startup file
from startup import *
#-----
import numpy as np
import matplotlib.pyplot as p
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
from astropy.coordinates import SkyCoord
from scipy import stats
from astropy.nddata import CCDData
import ccdproc
from ccdproc import ImageFileCollection, Combiner, combine
import glob
from astropy.wcs import WCS
from astropy.visualization import ZScaleInterval
zscale = ZScaleInterval()




%matplotlib widget

#Add useful show_image defintion 


def show_image(image,
               percl=99, percu=None, is_mask=False,
               figsize=(10, 10),
               cmap='viridis', log=False, clip=True,
               show_colorbar=True, show_ticks=True,
               fig=None, ax=None, input_ratio=None):
    """
    Show an image in matplotlib with some basic astronomically-appropriate stretching. 
    Taken from https://github.com/astropy/ccd-reduction-and-photometry-guide/blob/main/notebooks/convenience_functions.py

    Parameters
    ----------
    image
        The image to show
    percl : number
        The percentile for the lower edge of the stretch (or both edges if ``percu`` is None)
    percu : number or None
        The percentile for the upper edge of the stretch (or None to use ``percl`` for both)
    figsize : 2-tuple
        The size of the matplotlib figure in inches
    """
    if percu is None:
        percu = percl
        percl = 100 - percl

    if (fig is None and ax is not None) or (fig is not None and ax is None):
        raise ValueError('Must provide both "fig" and "ax" '
                         'if you provide one of them')
    elif fig is None and ax is None:
        if figsize is not None:
            # Rescale the fig size to match the image dimensions, roughly
            image_aspect_ratio = image.shape[0] / image.shape[1]
            figsize = (max(figsize) * image_aspect_ratio, max(figsize))

        fig, ax = p.subplots(1, 1, figsize=figsize)


    # To preserve details we should *really* downsample correctly and
    # not rely on matplotlib to do it correctly for us (it won't).

    # So, calculate the size of the figure in pixels, block_reduce to
    # roughly that,and display the block reduced image.

    # Thanks, https://stackoverflow.com/questions/29702424/how-to-get-matplotlib-figure-size
    fig_size_pix = fig.get_size_inches() * fig.dpi

    ratio = (image.shape // fig_size_pix).max()

    if ratio < 1:
        ratio = 1

    ratio = input_ratio or ratio

    reduced_data = block_reduce(image, ratio)

    if not is_mask:
        # Divide by the square of the ratio to keep the flux the same in the
        # reduced image. We do *not* want to do this for images which are
        # masks, since their values should be zero or one.
         reduced_data = reduced_data / ratio**2

    # Of course, now that we have downsampled, the axis limits are changed to
    # match the smaller image size. Setting the extent will do the trick to
    # change the axis display back to showing the actual extent of the image.
    extent = [0, image.shape[1], 0, image.shape[0]]

    if log:
        stretch = aviz.LogStretch()
    else:
        stretch = aviz.LinearStretch()

    norm = aviz.ImageNormalize(reduced_data,
                               interval=aviz.AsymmetricPercentileInterval(percl, percu),
                               stretch=stretch, clip=clip)

    if is_mask:
        # The image is a mask in which pixels should be zero or one.
        # block_reduce may have changed some of the values, so reset here.
        reduced_data = reduced_data > 0
        # Set the image scale limits appropriately.
        scale_args = dict(vmin=0, vmax=1)
    else:
        scale_args = dict(norm=norm)

    im = ax.imshow(reduced_data, origin='lower',
                   cmap=cmap, extent=extent, aspect='equal', **scale_args)

    if show_colorbar:
        # I haven't a clue why the fraction and pad arguments below work to make
        # the colorbar the same height as the image, but they do....unless the image
        # is wider than it is tall. Sticking with this for now anyway...
        # Thanks: https://stackoverflow.com/a/26720422/3486425
        fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        # In case someone in the future wants to improve this:
        # https://joseph-long.com/writing/colorbars/
        # https://stackoverflow.com/a/33505522/3486425
        # https://matplotlib.org/mpl_toolkits/axes_grid/users/overview.html#colorbar-whose-height-or-width-in-sync-with-the-master-axes


        

## Exploring Biases, Headers and Fits files using astropy

Lets set the paths and open a single bias files to plot up and explore. 

Begin by setting the correct path of the raw data.

In [ ]:
#setting the paths of the raw data.
#raw_path = add path

Use pyfits (astropy.io.fits) to open the first bias file. We will use this bias file explore. 

In [ ]:
# open the first bias image
#bias1 = pyfits.open()[0]

Once the bias is open explore/print the header

In [ ]:
#grab and then print the header 


Once you print the header grab the following information from the header to store in a Table 
filename, image type, object name, filter, exposure time, airmass

In [ ]:
#add code here to print specific peices of the header pieces of the header as above 



Now that you have obtained the the informaiton from the header, lets:
1. Plot the first bias (highly reccomend either using a zscale for the colorbar or vmin,vmax that is about 1 std -/+ from median )
2. Use the data to compute summary statistics about the bias (e.g., min, max, median, mean, std, )

In [ ]:
#--- insert code here that will compute basic statistics of the first bias and plot it. Do not use show image code just yet ---
#HINT for plotting use either -/+ 1 std for vmin/vmax or better use Zscale 





## Comparing Biases 

Now that you have explored the first bias; laod in the second bias and answer the question: 

Q : How different are the First and second biases?

In [ ]:
#--- insert code here that will compute basic statistics of the second bias and plot it. Do not use show image code just yet ---
#HINT for plotting use either -/+ 1 std for vmin/vmax or better use Zscale 


Q : How different are the First and second biases?

A: 

## Exploring the Night Directory and Stacking the Biases 

We can exlpore all of the files in the night directory using a very useful package called ccdproc (i.e. CCD Process). This is a pythonic version of the IRAF tasks called CCDProc. The command we will use to explore the directory is called ImageFileCollection. 

In [ ]:
# --- add information you would like from the keys --- 
#keys = [add keywords here]
ic1 = ImageFileCollection(raw_path, keywords=keys)
summary_tab = ic1.summary
summary_tab.show_in_notebook()

## Plot the airmass distribution 

It may be worth looking at the information in the headers. Lets plot the distribution of airmasses that we have sampled through out the night and answer the question : What do you notice about the airmass distirbution? 




In [ ]:
# add airmass distribtion here


Q: What do you notice about the airmass distirbution? 

A: 

## Stacking the Biases

Now our task is to look what happens when we combine all of the biases into a single median stacked bias which we will call the "Master Bias" Once you compute the median bias (across all biases) plot it and compare its statistics to the first bias. 

In [ ]:
# add code here to stack the biases using a median, compute the final statistics for the stacked bias and plot it.



    

Q: What do you notice about the "Master bias" comapred to a single bias frame? 

A:

## Measuring the Dark Current

The Dark Current Can be important. Lets check if it is or not. 

To do this we need to determine how different is the dark frame from the master bias frame *IN COUNTS* and then divide that by the exposure time of the dark frame. To detemrine if it matters we can then compare this against the Readnoise to determine the total exposure time required to achieve 1 electron per pixel in dark current. 

Q: A. How large is the bias current in electrons per second per pixel?; B. How does it compare to the readnoise, and C. How long does an exposure time 

For this excerise compute the bias current in the longest and shortest exposed dark frame! 



In [ ]:
#---- add code here
# open the longest dark image
from astropy.stats import sigma_clip
dark1 = pyfits.open(raw_path + 'dark0004.fits')[0]
diffs =  dark1.data - master_bias
filtered_diff= sigma_clip(diffs, sigma=3, maxiters=None,
                           cenfunc=np.median, masked=False, copy=False)

# compare it against the bias 

# compute the dark current accounting for the gain and darktime

#print out the amount of dark current, compare it to the read noise, and compute the exopsure time required to make the Dark Current larger than the read noise

print(np.mean(filtered_diff))
DC = abs(np.mean(diffs)) * bias1.header['GAIN'] / dark1.header['DARKTIME']
print('The expected Dark Current is %.2e electrons per second per pixel'%DC)
print('The Readnoise is %.2f electrons'%bias1.header['RDNOISE'])
print('Thus the exposure time required to make the DC larger than the read noise is %i seconds'%(bias1.header['RDNOISE']**2/ DC )) 
#NOTE  -- The square is b/c the RN = sqrt(N_electrons)!

In [ ]:
Q : 

A: 

## Stacking a Bandpass Specific Flat 

Now that we have a "Master Bias" from median stacking all of our bias frames, our next task is to stack all of the flats (using the same procedure as for the biases). 

To do this :
1. Choose a bandpass to work with
2. Plot and compute the statistics for a single flat frame of you choice
3. Median Stack all the *!bias subtracted!* flats in a bandpass of your choice 
4. Plot and compute the statistics for the median stacked flat frame in the bandpass of your chocie and compare against the single flat (from step 2)


In [ ]:
# --- Answer goes here





# Basic Reduction of an Object Frame 

Lets now :
1. Read in single object/science frame of your choice in the same bandpass you choose above.
2. Plot the object frame
3. Account for the bias (subtract) and flat (divide) and plot the 'finalized' image. HINT : the master should be normalized by the maximum and bias corrected 

In [ ]:
# --- Read in the object 


# ---grab the fits data 

#---plot the object data be sure to note in the title the object name; bandpass; exposure time







## "Clean" the image by account for the bias and flat

Start by accounting for the bias and flat on the object frame and then plot it up

In [ ]:
# "Clean" the object data by accounting for the flat and bias! 



In [ ]:
#plot the  "CLEANED" object data 








# Trim and fix the bad columns 

As you can tell, there is a "overscan" region (that contains the bias) and there are bad columns. We want to:
1. Trim the overscan region
2. Write some functions that can account for the bad columns 

In [ ]:
#---defintion to fix bad pixels-----
def fixpixels(image,badpixels):
    '''
    Code that will apply a 'fix pix' base routine on a series of bad columns.
    INPUT : 
    -Image data to be cleaned 
    -badpixels (Nx4 array) -- the [Xstart, Xend, Ystart, Yend] of the bad pixels/columns to be interoplated over 
    OUTPUT : Cleaned image data 

    '''

    
    M = np.full((np.shape(image.data)[0],np.shape(image.data)[1]),False)
    for j in range(len(badpixels)):
        M[badpixels[j][2]:badpixels[j][3],badpixels[j][0]:badpixels[j][1]] = True
    X = np.ma.array(image.data,mask=M)
    cleaned_image =  fixpix_baseroutine(X.data,X.mask)
    return cleaned_image


# --- base routine to fix pixels by interoplating over the bad/masked regions
def fixpix_baseroutine(data, mask, kind='linear'):
    '''
    Baserounte Code that will apply a fix the pixels by taking all masked vlaues in the array and doing a linear interpolation over the masked pixels 
    INPUT : Image data to be cleaned, mask from the array, kind = 'linear' which is the interoplation type
    OUTPUT : Cleaned image data 
    '''
    """Interpolate 2D array data in rows"""
    import numpy as np
    from scipy.interpolate import interp1d
    if data.shape != mask.shape: #mask and data in image must have same dimensions
        raise(ValueError)

    if not np.any(mask):
        return data # if it doesnt just return the data

    x = np.arange(0, data.shape[1])
    for row, mrow in zip(data, mask):
        if np.any(mrow):  # Interpolate if there's some pixel missing
            valid = (mrow == np.False_)
            invalid = (mrow == np.True_)

            itp = interp1d(x[valid], row[valid], kind=kind, copy=False) 
            row[invalid] = itp(x[invalid]).astype(row.dtype)
    return data


In [ ]:
#use the above codes to fix the bad pixels by clean_obj_fixpix = fixpixels(clean_obj_data)
# -- uncomment below
#badpixels = [[Xstart,Xend,Ystart,Yend], ...]
#trimed_clean_obj = fixpixels(image,badpixels)



Finally with the cleaned object data that whose pixels have been fixed please plot the final CCD reduced image 

In [ ]:
#plot the  "CLEANED" object data





